## Upload Video to YouTube Channel to prepare for uploading to Google Ads

In [22]:
#import httplib
import http.client as httplib
import httplib2
import os
import io
import random
import sys
import time
from argparse import Namespace

from apiclient.discovery import build
from apiclient.errors import HttpError
from apiclient.http import MediaFileUpload # this is for uploading video from local system
from googleapiclient.http import MediaIoBaseUpload # this is for GCS based upload
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import argparser, run_flow
from google.cloud import storage

In [9]:
# API setting & credentials
CLIENT_SECRETS_FILE = "client_secrets.json"
YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
MISSING_CLIENT_SECRETS_MESSAGE = "WARNING: Please configure OAuth 2.0"
VALID_PRIVACY_STATUSES = ("public", "private", "unlisted")
httplib2.RETRIES = 1

# Maximum number of times to retry before giving up.
MAX_RETRIES = 10

# Always retry when these exceptions are raised.
RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError, httplib.NotConnected,
  httplib.IncompleteRead, httplib.ImproperConnectionState,
  httplib.CannotSendRequest, httplib.CannotSendHeader,
  httplib.ResponseNotReady, httplib.BadStatusLine)

# Always retry when an apiclient.errors.HttpError with one of these status
# codes is raised.
RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

In [18]:
# helper functions
def resumable_upload(insert_request: MediaFileUpload) -> None:
    """
    Performs a resumable upload, with exponential backoff for retries.
    """
    response = None
    retry = 0
    while response is None:
        error = None
        try:
            print("Uploading file...")
            status, response = insert_request.next_chunk()
            if response is not None:
                if "id" in response:
                    # Use an f-string for modern, readable string formatting
                    print(f"Video id '{response['id']}' was successfully uploaded.")
                else:
                    # Use sys.exit() for cleaner script termination
                    sys.exit(
                        f"The upload failed with an unexpected response: {response}"
                    )

        # Use 'except Exception as e' syntax
        except HttpError as e:
            if e.resp.status in RETRIABLE_STATUS_CODES:
                error = f"A retriable HTTP error {e.resp.status} occurred:\n{e.content}"
            else:
                # Re-raise the exception if it's not a retriable error
                raise
        except RETRIABLE_EXCEPTIONS as e:
            error = f"A retriable error occurred: {e}"

        if error is not None:
            print(error)
            retry += 1
            if retry > MAX_RETRIES:
                sys.exit("No longer attempting to retry.")

            # Implement exponential backoff with jitter
            max_sleep = 2**retry
            sleep_seconds = random.random() * max_sleep

            # Correctly format the string inside the print() function call
            print(f"Sleeping {sleep_seconds:.2f} seconds and then retrying...")
            time.sleep(sleep_seconds)
    
    return response['id']


def get_authenticated_service(args):
    flow = flow_from_clientsecrets(
        CLIENT_SECRETS_FILE,
        scope=YOUTUBE_UPLOAD_SCOPE,
        message=MISSING_CLIENT_SECRETS_MESSAGE,
    )

    storage = Storage("%s-oauth2.json" % sys.argv[0])
    credentials = storage.get()

    if credentials is None or credentials.invalid:
        credentials = run_flow(flow, storage, args)

    return build(
        YOUTUBE_API_SERVICE_NAME,
        YOUTUBE_API_VERSION,
        http=credentials.authorize(httplib2.Http()),
    )


def initialize_upload(youtube, options):
    tags = None
    if options.keywords:
        tags = options.keywords.split(",")
    
    body = dict(
        snippet=dict(
            title=options.title,
            description=options.description,
            tags=tags,
            categoryId=options.category,
        ),
        status=dict(privacyStatus=options.privacyStatus),
    )  
    
    if options.use_gcs:
        print("Uploading from GCS directly...")
        storage_client = storage.Client()
        bucket = storage_client.bucket(options.bucket_name)
        blob = bucket.blob(options.source_blob_name)

        # 2. Create a readable stream from the GCS object
        # This is an in-memory, file-like object.
        fh = io.BytesIO()
        blob.download_to_file(fh)
        fh.seek(0) # IMPORTANT: Rewind the stream to the beginning

        # 3. Use MediaIoBaseUpload with the stream
        # Notice we are not using MediaFileUpload anymore.
        media_body = MediaIoBaseUpload(
            fh, 
            mimetype='video/mp4', # Specify the mimetype
            chunksize=-1, 
            resumable=True
        )
        
    else:
        media_body = MediaFileUpload(options.file, chunksize=-1, resumable=True)

    # Call the API's videos.insert method to create and upload the video.
    insert_request = youtube.videos().insert(
        part=",".join(body.keys()),
        body=body,
        # The chunksize parameter specifies the size of each chunk of data, in
        # bytes, that will be uploaded at a time. Set a higher value for
        # reliable connections as fewer chunks lead to faster uploads. Set a lower
        # value for better recovery on less reliable connections.
        #
        # Setting "chunksize" equal to -1 in the code below means that the entire
        # file will be uploaded in a single HTTP request. (If the upload fails,
        # it will still be retried where it left off.) This is usually a best
        # practice, but if you're using Python older than 2.6 or if you're
        # running on App Engine, you should set the chunksize to something like
        # 1024 * 1024 (1 megabyte).
        media_body=media_body,
    )

    response_id = resumable_upload(insert_request)
    return response_id


In [27]:
# Upload parameters
args = Namespace(
    #file='sample_0.mp4', # this is for when uploading from local disk - when uploading from GCS don't need to specify this
    title='yuan test', 
    description="yuan_test",
    category="22",
    privacyStatus="unlisted",
    logging_level="INFO",
    noauth_local_webserver=True, #when setting to true, no redirection will happen
    #auth_host_port=[8080, 8090],
    #auth_host_name='localhost',
    keywords="yuan test",
    use_gcs=True,
    bucket_name=GCS_BUCKET,
    source_blob_name="generated_video/10832930450041060730/sample_0.mp4"
)

In [28]:
youtube = get_authenticated_service(args)

#somehow i had to create a "desktop app credential" here. the web app credential which i'm using for GAds API dont work

In [29]:
response_id = initialize_upload(youtube, args)

Uploading from GCS directly...
Uploading file...
Video id '28CVAirUnww' was successfully uploaded.
